# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
import plotly.offline as pyo
import plotly.graph_objs as go
from math import sqrt
#physical_devices = tf.config.experimental.list_physical_devices()
#print("Num GPUs:", physical_devices)
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Importing Data

In [2]:
#confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [3]:
confirmed_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,7653,8145,8676,9216,9998,10582,11173,11831,12456,13036,13659,14525,15205,15750,16509,17267,18054,18969,19551,20342,20917,21459,22142,22890,23546,24102,24766,25527,26310,26874,27532,27878,28424,28833,29157,29481,29640,30175,30451,30616
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,949,964,969,981,989,998,1004,1029,1050,1076,1099,1122,1137,1143,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385,1416,1464,1521,1590,1672,1722,1788,1838,1891,1962,1995,2047,2114,2192,2269,2330
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,7377,7542,7728,7918,8113,8306,8503,8697,8857,8997,9134,9267,9394,9513,9626,9733,9831,9935,10050,10154,10265,10382,10484,10589,10698,10810,10919,11031,11147,11268,11385,11504,11631,11771,11920,12076,12248,12445,12685,12968
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,761,762,762,762,762,762,763,763,763,763,764,764,764,765,844,851,852,852,852,852,852,852,852,852,853,853,853,853,854,854,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,52,52,58,60,61,69,70,70,71,74,81,84,86,86,86,86,86,86,88,91,92,96,113,118,130,138,140,142,148,155,166,172,176,183,186,189,197,212,212,259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,251,251,251,251,251,251,299,441,443,458,463,479,483,484,484,484,485,499,499,513,513,514,611,632,639,659,661,662,671,683,688,693,698,698,702,707,710,711,712,713
262,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,167,184,197,209,212,222,233,249,256,278,283,310,323,354,399,419,453,469,482,484,496,524,560,591,632,705,728,844,885,902,909,919,922,941,967,992,1015,1076,1089,1103
263,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,11,34,34,78,78,87,87,87,87,87,87,106,106,106,132,132,132,132,141,141,141,141,162,162,163,176,176,176,197,197,210,210,247,247,247,265,265,272,272,272
264,NaN,Tajikistan,38.861034,71.276093,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1936,2140,2350,2551,2738,2929,3100,3266,3424,3563,3686,3807,3930,4013,4100,4191,4289,4370,4453,4529,4609,4690,4763,4834,4902,4971,5035,5097,5160,5221,5279,5338,5399,5457,5513,5567,5630,5691,5747,5799


# Date Pre-Processing

In [4]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))  # last days to predict  

confirmed_df = confirmed_df.groupby("Country/Region").sum().reset_index()
Country_confirmed =  confirmed_df['Country/Region'].values
confirmed_df = confirmed_df.drop(['Lat','Long','Country/Region'], axis=1)
confirmed_df = confirmed_df[confirmed_df.columns].T
confirmed_df.columns = Country_confirmed                                    
compare_confirmed_df=confirmed_df.iloc[-DAYS_TO_PREDICT-1:,:]                 # last days to predict
confirmed_df= confirmed_df.iloc[:-DAYS_TO_PREDICT,:]                        # last days to predict

Date_index = confirmed_df.index
confirmed_df = confirmed_df.diff().fillna(0)
compare_confirmed_df = compare_confirmed_df.diff().dropna(0)
confirmed_df = confirmed_df.abs()

confirmed_df


Enter Number Of Days You Want to Predict : 5


,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo (Brazzaville),...,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan*,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,US,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
1/22/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/23/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
1/24/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/25/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,486.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/26/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,669.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6/18/20,658.0,66.0,117.0,1.0,11.0,0.0,1958.0,665.0,18.0,20.0,338.0,0.0,469.0,3803.0,0.0,625.0,104.0,0.0,25.0,0.0,814.0,33.0,0.0,22765.0,0.0,132.0,0.0,24.0,0.0,31.0,1.0,0.0,386.0,41.0,0.0,4475.0,36.0,2119.0,13.0,0.0,...,106.0,94.0,0.0,23.0,257.0,1.0,8.0,23.0,3478.0,17.0,585.0,23.0,0.0,16.0,1481.0,13.0,9.0,1.0,58.0,0.0,6.0,0.0,3.0,0.0,4.0,1304.0,27762.0,9.0,847.0,388.0,1218.0,1.0,85.0,97.0,7.0,45.0,0.0,7.0,4.0,62.0
6/19/20,346.0,50.0,119.0,0.0,6.0,0.0,2060.0,459.0,2.0,48.0,438.0,0.0,486.0,3243.0,0.0,676.0,128.0,0.0,53.0,1.0,977.0,99.0,10.0,54771.0,0.0,81.0,1.0,0.0,0.0,25.0,0.0,774.0,437.0,0.0,4.0,6290.0,0.0,3185.0,0.0,0.0,...,164.0,93.0,0.0,26.0,142.0,14.0,2.0,0.0,3825.0,34.0,307.0,3.0,296.0,16.0,2033.0,35.0,0.0,0.0,59.0,0.0,5.0,0.0,8.0,0.0,14.0,1214.0,31527.0,14.0,922.0,393.0,1350.0,3.0,179.0,108.0,7.0,75.0,0.0,10.0,14.0,16.0
6/20/20,546.0,53.0,127.0,0.0,4.0,0.0,1634.0,551.0,50.0,52.0,471.0,0.0,415.0,3240.0,0.0,603.0,74.0,0.0,0.0,0.0,1036.0,0.0,0.0,34666.0,0.0,117.0,1.0,1.0,0.0,15.0,0.0,972.0,448.0,81.0,0.0,5355.0,59.0,3067.0,37.0,0.0,...,144.0,94.0,0.0,11.0,218.0,10.0,6.0,36.0,4966.0,18.0,363.0,0.0,264.0,10.0,515.0,8.0,11.0,0.0,61.0,0.0,1.0,0.0,6.0,0.0,10.0,1248.0,32718.0,8.0,860.0,388.0,1295.0,6.0,207

## Enter countries you want to forecast


In [5]:
i = int(input('Number of countries you want to consider : '))
countries = [input('Enter country You Want to forecast : ') for i in range(i)]

Number of countries you want to consider : 3
Enter country You Want to forecast : Germany
Enter country You Want to forecast : Italy
Enter country You Want to forecast : China


In [6]:
countries

['Germany', 'Italy', 'China']

In [7]:
confirmed_df = confirmed_df.loc[:,countries]   
compare_confirmed_df = compare_confirmed_df.loc[:,countries]  # last days to predict 

In [8]:
confirmed_df

,Germany,Italy,China
1/22/20,0.0,0.0,0.0
1/23/20,0.0,0.0,95.0
1/24/20,0.0,0.0,277.0
1/25/20,0.0,0.0,486.0
1/26/20,0.0,0.0,669.0
...,...,...,...
6/18/20,1213.0,331.0,36.0
6/19/20,482.0,148.0,0.0
6/20/20,371.0,264.0,59.0
6/21/20,602.0,224.0,19.0


# Scalling the data

In [9]:
sc = RobustScaler()
confirmed_df_scaled = sc.fit_transform(confirmed_df)
confirmed_df_scaled = pd.DataFrame(confirmed_df_scaled)
confirmed_df_scaled

,0,1,2
0,-0.314322,-0.260580,-0.252577
1,-0.314322,-0.260580,0.237113
2,-0.314322,-0.260580,1.175258
3,-0.314322,-0.260580,2.252577
4,-0.314322,-0.260580,3.195876
...,...,...,...
148,0.454373,-0.130878,-0.067010
149,-0.008872,-0.202586,-0.252577
150,-0.079214,-0.157132,0.051546
151,0.067174,-0.172806,-0.154639


# Enter the number of days you want to predict

In [10]:
# DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : ')) # when future forecast is done. 
predict_confirm_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_confirm_index = predict_confirm_index.strftime('%Y/%m/%d')

# Creating class for building model

In [11]:
class Corona_Confirmed_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
            a = dataset[i:(i+time_steps), column]
            dataX.append(a)
            dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        confirm_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
        
        optimizer = keras.optimizers.Adadelta(lr=0.001)
        
        confirm_model.compile(loss=['mae'],optimizer=optimizer)   
                
        confirm_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)
        

        confirm_test_seq  = trainX[-1:]
        confirm_test_seq = confirm_test_seq
        confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq), time_steps, 1)
        confirm_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            confirm_pred = confirm_model.predict(confirm_test_seq)
            confirm_preds.append(confirm_pred)
            confirm_new_seq = confirm_pred
            confirm_test_seq = confirm_new_seq
            confirm_test_seq = confirm_test_seq.reshape(len(confirm_test_seq ), time_steps, 1)

        

        return confirm_preds

# Calling the class and predicting the cases for every individual country

In [12]:
NN = Corona_Confirmed_model()
confirm_predictions = []
for i in range(len(countries)):
  result = NN.Model(confirmed_df_scaled,i) 
  confirm_predictions.append(result)
confirm_predictions        

[[array([[[-0.2000331]]], dtype=float32),
  array([[[-0.27865452]]], dtype=float32),
  array([[[-0.3014042]]], dtype=float32),
  array([[[-0.30916765]]], dtype=float32),
  array([[[-0.31194478]]], dtype=float32)],
 [array([[[-0.18010719]]], dtype=float32),
  array([[[-0.19862218]]], dtype=float32),
  array([[[-0.21365625]]], dtype=float32),
  array([[[-0.22594324]]], dtype=float32),
  array([[[-0.23604085]]], dtype=float32)],
 [array([[[0.01383077]]], dtype=float32),
  array([[[0.01103138]]], dtype=float32),
  array([[[0.00360775]]], dtype=float32),
  array([[[-0.01031718]]], dtype=float32),
  array([[[-0.05513616]]], dtype=float32)]]

# Processing the predicted data

In [13]:
confirm_predictions = (np.array(confirm_predictions).transpose())
confirm_predictions = confirm_predictions.reshape(((DAYS_TO_PREDICT),len(countries)))
confirm_predictions = sc.inverse_transform(confirm_predictions)
confirm_predictions = pd.DataFrame(confirm_predictions)
confirm_predictions = confirm_predictions.abs()
confirm_predictions['Dates'] = predict_confirm_index
confirm_predictions.set_index('Dates', inplace = True)
confirm_predictions.columns = countries
confirm_predictions.head()

,Germany,Italy,China
Dates,,,
2020/06/23,180.347778,205.366455,51.683170
2020/06/24,56.283173,158.116180,51.140087
2020/06/25,20.384155,119.749268,49.699905
2020/06/26,8.133453,88.392883,46.998466
2020/06/27,3.751129,62.623779,38.303585


# Visualizing The Confirmed Cases For A Country As Per The User's Choice

### List of Infected Countries

In [14]:
Infected_Countries = pd.DataFrame(countries)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries

,Countries
0,Germany
1,Italy
2,China


### Select The Country

In [15]:
Country =input('Enter the name of the country from above list you would like to see the daily cases till now : ')

Enter the name of the country from above list you would like to see the daily cases till now : Italy


## Till Date Confirm Cases on daily basis of coronavirus

In [16]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Confirmed Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                    xaxis_title = "Dates",
                    yaxis_title="Number Of Cases",
                     )
fig.add_trace(go.Scatter(x=confirmed_df.index, 
                         y=confirmed_df[Country],
                         mode='lines+markers',
                         line=dict(color='blue', width=2)))

## Predictions of confirmed cases of coronavirus

In [17]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Confirmed Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
    xaxis_title = "Dates",
    yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=confirm_predictions.index, 
                         y=confirm_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='yellow', width=2)))

In [18]:
#pd.concat((confirmed_df,confirm_predictions))

In [19]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Confirmed Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
    xaxis_title = "Dates",
    yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=pd.concat((confirmed_df,confirm_predictions)).index, 
                         y=pd.concat((confirmed_df,confirm_predictions))[Country],
                         mode='lines+markers',
                         line=dict(color='red', width=2)))

## Root mean square error between last days predictions(only for comment last days to predict)

In [20]:
compare_confirmed_df

,Germany,Italy,China
6/23/20,712.0,113.0,29.0
6/24/20,391.0,577.0,20.0
6/25/20,500.0,296.0,28.0
6/26/20,665.0,255.0,24.0
6/27/20,422.0,175.0,18.0


In [21]:
confirm_predictions

,Germany,Italy,China
Dates,,,
2020/06/23,180.347778,205.366455,51.683170
2020/06/24,56.283173,158.116180,51.140087
2020/06/25,20.384155,119.749268,49.699905
2020/06/26,8.133453,88.392883,46.998466
2020/06/27,3.751129,62.623779,38.303585


In [22]:
for country in countries:
  realVals = compare_confirmed_df[country]
  predictedVals = confirm_predictions[country]
  rmse = sqrt(mean_squared_error(realVals, predictedVals))
  print('RMSE for {} :'.format(country),rmse )

RMSE for Germany : 496.21095834719307
RMSE for Italy : 226.0291500330374
RMSE for China : 24.067746729389224
